<a href="https://colab.research.google.com/github/rkmaccha/ContentBased_RecommendationSystems/blob/main/Sklearn_Pipeline_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

USING PIPELINE PROCESS FOR ML PROBLEM

In [ ]:
from sklearn import impute
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/MachineLearning_Journey/dataset/titanic_train.csv")

In [ ]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
df.drop(columns=['Name','Ticket','Cabin','PassengerId'], inplace = True)

In [ ]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [ ]:
X = df.drop(columns=['Survived'],axis=1)
y = df['Survived']

In [ ]:
X.shape

(891, 7)

In [ ]:
y.shape

(891,)

In [ ]:
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2, random_state=42)

In [ ]:
y_train.sample(5)

35     0
412    1
547    1
546    1
73     0
Name: Survived, dtype: int64

In [ ]:
X_train.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5,S
733,2,male,23.0,0,0,13.0,S


In [ ]:
# Imputation Transformer
trf_01=  ColumnTransformer([('impute_age',SimpleImputer(),[2]),('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])],remainder='passthrough')

In [ ]:
# One Hot Encoder
trf_02 = ColumnTransformer([('ohe_sex_emabarked',OneHotEncoder(sparse=False,handle_unknown='ignore'),[1,6])],remainder ='passthrough')

In [ ]:
from pandas.core.base import SelectionMixin
# Scaling
trf_03 = ColumnTransformer([('Scale',MinMaxScaler(),slice(0,10))])

In [ ]:
# Feature Selection
trf_04 = SelectKBest(score_func=chi2,k=8)

In [ ]:
# train Model
trf_05 = DecisionTreeClassifier()

In [ ]:
pipe = Pipeline([('trf1',trf_01),('trf2',trf_02),('trf3',trf_03),('trf4',trf_04),('trf5',trf_05)])

In [ ]:
make_pipe = make_pipeline(trf_01,trf_02,trf_03,trf_04,trf_05)

In [ ]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_emabarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('Scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),

In [ ]:
# Display Pipeline Flow
from sklearn import set_config
set_config(display='diagram')

In [ ]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_emabarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('Scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7fbd2d987a70>)),
                ('trf5', DecisionTreeClassifier())])

In [ ]:
y_pred = pipe.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.6256983240223464

============== Debugging the pipeline ==========================

In [ ]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_emabarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('Scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=8, score_func=<function chi2 at 0x7fbd2d987a70>),
 'trf5': DecisionTreeClassifier()}

In [ ]:
pipe.named_steps['trf1'].transformers_

[('impute_age', SimpleImputer(), [2]),
 ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6]),
 ('remainder', 'passthrough', [0, 1, 3, 4, 5])]

In [ ]:
pipe.named_steps['trf1'].transformers_[0][1].statistics_

array([29.49884615])

In [ ]:
pipe.named_steps['trf1'].transformers_[1][1].statistics_

array(['S'], dtype=object)

==============================================================

##Cross-Validatation using Pipeline

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,X_train,y_train,cv=5,scoring="accuracy").mean()

0.6391214419383433

## GridSearch using Pipeline

In [ ]:
# gridsearchcv
params = { 'trf5__max_depth':[1,2,3,4,5,None]}

In [ ]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe,params,cv=5,scoring="accuracy")


In [ ]:
grid.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__memory', 'estimator__steps', 'estimator__verbose', 'estimator__trf1', 'estimator__trf2', 'estimator__trf3', 'estimator__trf4', 'estimator__trf5', 'estimator__trf1__n_jobs', 'estimator__trf1__remainder', 'estimator__trf1__sparse_threshold', 'estimator__trf1__transformer_weights', 'estimator__trf1__transformers', 'estimator__trf1__verbose', 'estimator__trf1__verbose_feature_names_out', 'estimator__trf1__impute_age', 'estimator__trf1__impute_embarked', 'estimator__trf1__impute_age__add_indicator', 'estimator__trf1__impute_age__copy', 'estimator__trf1__impute_age__fill_value', 'estimator__trf1__impute_age__missing_values', 'estimator__trf1__impute_age__strategy', 'estimator__trf1__impute_age__verbose', 'estimator__trf1__impute_embarked__add_indicator', 'estimator__trf1__impute_embarked__copy', 'estimator__trf1__impute_embarked__fill_value', 'estimator__trf1__impute_embarked__missing_values', 'estimator__trf1__impute_embarked__strategy', 'estimato

In [ ]:
grid.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_emabarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('Scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x7fbd2d987a70>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [ ]:
  grid.best_score_

0.6391214419383433

In [ ]:
grid.best_params_

{'trf5__max_depth': 2}

## Exporting the Pipeline

In [ ]:
import pickle
pickle.dump(pipe,open('/content/drive/MyDrive/MachineLearning_Journey/Models_save/pipe.pkl','wb'))

## Importing the model Pipeline

In [ ]:
import pickle
import numpy as np

In [ ]:
pipe = pickle.load(open('/content/drive/MyDrive/MachineLearning_Journey/Models_save/pipe.pkl','rb'))

In [ ]:
test_input = np.array([2,'male',31.0,0,0,10.5,'S'],dtype=object).reshape(1,7)

In [ ]:
pipe.predict(test_input)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  "X does not have valid feature names, but"


array([0])